In [186]:
from PIL import Image
import pytesseract
from pytesseract import Output
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import re

#### To make your own ticker array. 

1. Change the name and save the file to load it later.

#### Creating a folder structure for all the screenshots.

1. Rohit_data   >>>   Companies   >>>   All tickers   >>>   In each ticker two folders with suppliers and customers. 

In [12]:
supplier_columns = ['Company', 'Industry', 'Mkt Cap (M)', 'Relationship Size (M)', '% Cost',
           'Cost Cat', '% Supplier Rev', 'Source', 'Size Source']
customer_columns = ['Company', 'Industry', 'Mkt Cap (M)', 'Relationship Size (M)', '% Rev',
           '% Customer Cost', 'Customer Cost Cat', 'Source', 'Size Source']

# Read files in folder

In [216]:
import os
rootdir = '/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/SKYS/Suppliers'
def listdirs(rootdir):
    file_list = []
    for file in os.listdir(rootdir):
        print(rootdir + "/"+file)
        file_list.append(rootdir + "/"+file)
    return file_list
        

In [218]:
listdirs(rootdir)

True

In [206]:
# Define the replacement function
def replace_em_dash(cell_value):
    if isinstance(cell_value, str) and '—' in str(cell_value):
        return '--'
    else:
        return cell_value

# Define the replacement function
def replace_no_numbers(cell_value):
    if pd.notna(cell_value) and not bool(re.search(r'\d', str(cell_value))):
        return '--'
    return cell_value

In [201]:
def supplier_file(data):
    df = pd.DataFrame(data)
    df = df[['Company', 'Industry', 'Mkt Cap (M)', 'Relationship Size (M)', '% Cost',
           'Cost Cat', 'Supplier Rev', 'Source']]
    df["Source"] = df["Source"].str.extract(r'(\d{4})')
    name = df['Company'][0]
    df = df.rename(columns = {"Company":"Supplier Name"})
    df.insert(0, "Firm Name", name)
    df = df.iloc[1:]
    
    # replace em dash with "--"
    df = df.applymap(replace_em_dash)
    # Mk Cap, Relationship Size, % Cost, % Supplier Rev has to have numerical values if not "--"
    df[['Mkt Cap (M)', 'Relationship Size (M)', '% Cost','Supplier Rev']] = df[['Mkt Cap (M)', 
                                                                                  'Relationship Size (M)', 
                                                                                  '% Cost',
                                                                                  'Supplier Rev']].applymap(replace_no_numbers)

#     customer_dict = df.set_index('Company').agg(list, axis=1).to_dict()
#     arr = np.array([[ticker,name,customer_dict]])
    return df

## Load ticker files

In [211]:
tickers = np.load('/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/thao_tickers.npy', allow_pickle=True)

In [221]:
tickers = tickers[(tickers != "DSKE")&(tickers != "MCFT")]

In [228]:
# checked = TIKK, BOOT, AXTA, "FND","RSTRF","LCINQ","VSTO"
for ticker in tickers:
    rootdir = "/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/" + ticker + "/Suppliers"
    file_list = listdirs(rootdir)
    if file_list == []:
        continue    
    df = pd.DataFrame()
    for path in file_list:
        data = pd.read_csv(path)
#         df.append(supplier_file(data))
        df = pd.concat([df, supplier_file(data)], axis=0)
    
    # long name will end with "..."
    dot_pattern = r'(\.|\.\.|\.\.\.|\s\.|\s\.\.|\s\.\.\.)$'
    df['Supplier Name'] = df['Supplier Name'].str.replace(dot_pattern, '...', regex=True)
    df['Industry'] = df['Industry'].str.replace(dot_pattern, '...', regex=True)
    # treat long name of Firm Name
    if sum(df['Firm Name'].str.contains(dot_pattern, '...', regex=True))>0:
        df['Firm Name'] = df['Firm Name'].str.replace(dot_pattern, '...', regex=True)
        not_matching_pattern = ~df['Firm Name'].str.contains(dot_pattern)
        df.loc[not_matching_pattern, 'Firm Name'] = df.loc[not_matching_pattern, 'Firm Name'] + '...'    
    df = df.drop_duplicates(subset=['Supplier Name', 'Industry','Source']).reset_index(drop=True)
    df.insert(0, "ticker", ticker)
    df.to_csv("/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/" + ticker +"/suppliers.csv",index = False)        
    

/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/FRPT/Suppliers/S 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/VWR/Suppliers/S 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/BSGM/Suppliers/S 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/ANN/Suppliers/S 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/VIVE/Suppliers/S 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/CHRS/Suppliers/S 1.csv


/var/folders/5d/9jrnkbxx5bb_l2v7b5pxxnfm0000gn/T/ipykernel_82673/1468718567.py:18: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if sum(df['Firm Name'].str.contains(dot_pattern, '...', regex=True))>0:
/var/folders/5d/9jrnkbxx5bb_l2v7b5pxxnfm0000gn/T/ipykernel_82673/1468718567.py:18: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if sum(df['Firm Name'].str.contains(dot_pattern, '...', regex=True))>0:
/var/folders/5d/9jrnkbxx5bb_l2v7b5pxxnfm0000gn/T/ipykernel_82673/1468718567.py:18: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if sum(df['Firm Name'].str.contains(dot_pattern, '...', regex=True))>0:
/var/folders/5d/9jrnkbxx5bb_l2v7b5pxxnfm0000gn/T/ipykernel_82673/1468718567.py:18: UserWarning: This pattern is interpreted as a r

/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/MPG/Suppliers/S 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/MRMD/Suppliers/S 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/MTRAF/Suppliers/S 2.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/MTRAF/Suppliers/S 3.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/MTRAF/Suppliers/S 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/BOOT/Suppliers/S 2.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/BOOT/Suppliers/S 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/FGEN/Suppliers/S 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/NVRO/Suppliers/S 1.csv


/var/folders/5d/9jrnkbxx5bb_l2v7b5pxxnfm0000gn/T/ipykernel_82673/1468718567.py:18: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if sum(df['Firm Name'].str.contains(dot_pattern, '...', regex=True))>0:
/var/folders/5d/9jrnkbxx5bb_l2v7b5pxxnfm0000gn/T/ipykernel_82673/1468718567.py:18: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if sum(df['Firm Name'].str.contains(dot_pattern, '...', regex=True))>0:
/var/folders/5d/9jrnkbxx5bb_l2v7b5pxxnfm0000gn/T/ipykernel_82673/1468718567.py:18: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if sum(df['Firm Name'].str.contains(dot_pattern, '...', regex=True))>0:
/var/folders/5d/9jrnkbxx5bb_l2v7b5pxxnfm0000gn/T/ipykernel_82673/1468718567.py:18: UserWarning: This pattern is interpreted as a r

/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/MEVI/Suppliers/MEVI S 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/CRBP/Suppliers/S 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/SIEN/Suppliers/S 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/DIIBF/Suppliers/S 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/NMRD/Suppliers/S 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/AXTA/Suppliers/S 2.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/AXTA/Suppliers/S 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/TANH/Suppliers/S 1.csv


...

/var/folders/5d/9jrnkbxx5bb_l2v7b5pxxnfm0000gn/T/ipykernel_82673/1468718567.py:18: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if sum(df['Firm Name'].str.contains(dot_pattern, '...', regex=True))>0:
/var/folders/5d/9jrnkbxx5bb_l2v7b5pxxnfm0000gn/T/ipykernel_82673/1468718567.py:18: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if sum(df['Firm Name'].str.contains(dot_pattern, '...', regex=True))>0:
/var/folders/5d/9jrnkbxx5bb_l2v7b5pxxnfm0000gn/T/ipykernel_82673/1468718567.py:18: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if sum(df['Firm Name'].str.contains(dot_pattern, '...', regex=True))>0:
/var/folders/5d/9jrnkbxx5bb_l2v7b5pxxnfm0000gn/T/ipykernel_82673/1468718567.py:18: UserWarning: This pattern is interpreted as a r

/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/CBIO/Suppliers/S 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/CKH/Suppliers/S 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/ULBI/Suppliers/S 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/KURA/Suppliers/S 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/CTMX/Suppliers/S 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/HOME/Suppliers/S 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/NBRV/Suppliers/S 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/AVTX/Suppliers/S 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/HPE/Suppliers/S 2.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/HPE/Suppliers/S 3.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/HPE/Suppliers/S 1.csv
/Us

/var/folders/5d/9jrnkbxx5bb_l2v7b5pxxnfm0000gn/T/ipykernel_82673/1468718567.py:18: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if sum(df['Firm Name'].str.contains(dot_pattern, '...', regex=True))>0:
/var/folders/5d/9jrnkbxx5bb_l2v7b5pxxnfm0000gn/T/ipykernel_82673/1468718567.py:18: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if sum(df['Firm Name'].str.contains(dot_pattern, '...', regex=True))>0:
/var/folders/5d/9jrnkbxx5bb_l2v7b5pxxnfm0000gn/T/ipykernel_82673/1468718567.py:18: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if sum(df['Firm Name'].str.contains(dot_pattern, '...', regex=True))>0:
/var/folders/5d/9jrnkbxx5bb_l2v7b5pxxnfm0000gn/T/ipykernel_82673/1468718567.py:18: UserWarning: This pattern is interpreted as a r

/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/ANAB/Suppliers/S 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/GNST/Suppliers/S 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/IOTS/Suppliers/S 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/MYOK/Suppliers/S 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/DLTH/Suppliers/S 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/XCOMQ/Suppliers/S 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/WVE/Suppliers/S 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/VYGR/Suppliers/S 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/AFIIQ/Suppliers/S 2.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/AFIIQ/Suppliers/S 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/AXSM/Suppliers/S 1.c

/var/folders/5d/9jrnkbxx5bb_l2v7b5pxxnfm0000gn/T/ipykernel_82673/1468718567.py:18: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if sum(df['Firm Name'].str.contains(dot_pattern, '...', regex=True))>0:
/var/folders/5d/9jrnkbxx5bb_l2v7b5pxxnfm0000gn/T/ipykernel_82673/1468718567.py:18: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if sum(df['Firm Name'].str.contains(dot_pattern, '...', regex=True))>0:


In [226]:
np.where(tickers=="DSKE")

(array([], dtype=int64),)

In [205]:
df

,ticker,Firm Name,Supplier Name,Industry,Mkt Cap (M),Relationship Size (M),% Cost,Cost Cat,Supplier Rev,Source
0,ACB,Aurora Cannabis Inc,Strainprint Technologies Ltd,--,--,--,--,--,--,NaN
1,ACB,Aurora Cannabis Inc,PAX Labs Inc,--,--,--,--,--,--,NaN
2,ACB,Aurora Cannabis Inc,EnWave Corp,Machinery,40.90,--,--,--,--,2020
3,ACB,Aurora Cannabis Inc,CTT Pharmaceutical Holdings Inc,Biotech & Pharma,0.28,--,--,--,--,NaN
4,ACB,Aurora Cannabis Inc,Xcel Energy Inc,Electric Utilities,"44,883.36",--,--,--,--,2021
5,ACB,Aurora Cannabis Inc,Valens Co Inc/The,Tobacco & Cannabis,--,--,--,--,--,NaN
6,ACB,Aurora Cannabis Inc,Fortis Inc/Canada,Electric Utilities,"27,221.91",--,--,--,--,2021
7,ACB,Aurora Cannabis Inc,American Software Inc/GA,Software,471.78,--,--,--,--,NaN
8,ACB,Aurora Cannabis Inc,WEC Energy Group Inc,Electric Utilities,"36,732.28",--,--,--,--,2021


## Convert to Dict